# STRUCTURAL ANALYSIS

In [12]:
import pandas as pd
#import structural_analysis
import statistics
#import structure_validation
import importlib
from scipy import stats
import math
import numpy as np
from pathlib import Path  
import vcf
import matplotlib.patches as mpatches


import matplotlib.pyplot as plt
import seaborn as sns




SyntaxError: invalid syntax (3509021383.py, line 17)

In [4]:
#df_allsp = pd.read_pickle(r'C:\Users\Adam Martin\OneDrive - University of Dundee\Honours Project\Jalview and Chimera using the alignment from PF00026 first from linux\Uniprot_Sequences_For_Alignment_To_Allow_VarAlign_TO\Uniprot_Annotated_Alignment127_PFAM.sto_prointvar_structure_table.p.gz')
import pyarrow
df_allsp = pd.read_pickle('/homes/2414054/Honours/231006a-AM-honours-project-aps/varalign-data-PF00026/pickles/PF00026.22_swissprot.sto_prointvar_structure_table.p.gz') # GETS CONSENSUS COLUMNS FROM MSA (OCCUPANCY > 0.5)


In [5]:
df_allsp

,ATOM_A,ATOM_B,Alignment_column_A,Alignment_column_B,B_iso_or_equiv_A,B_iso_or_equiv_B,CATH_dbAccessionId_A,CATH_dbAccessionId_B,CATH_regionEnd_A,CATH_regionEnd_B,...,pdbx_formal_charge_B,type_symbol_A,type_symbol_B,interaction_type,protein_topology,pfam_domain_topology,cath_domain_topology,cath_hierarchical_topology,scop_domain_topology,polymer_topology
0,C,N,230.0,232.0,16.675000,12.560000,2.40.70.10,2.40.70.10,214.0,214.0,...,?,N,N,Protein-Protein,Homoprotein,Homodomain (Self),Homodomain,level_4,Homodomain,Intrapolymer
1,O,N,177.0,232.0,34.367500,12.560000,2.40.70.10,2.40.70.10,214.0,214.0,...,?,N,N,Protein-Protein,Homoprotein,Homodomain (Self),Homodomain,level_4,Homodomain,Intrapolymer
2,O,O,230.0,311.0,16.675000,18.305714,2.40.70.10,2.40.70.10,214.0,214.0,...,?,N,N,Protein-Protein,Homoprotein,Homodomain (Self),Homodomain,level_4,Homodomain,Intrapolymer
3,O,N,300.0,311.0,47.157500,18.305714,2.40.70.10,2.40.70.10,214.0,214.0,...,?,N,N,Protein-Protein,Homoprotein,Homodomain (Self),Homodomain,level_4,Homodomain,Intrapolymer
6,NE2,O,229.0,311.0,16.737778,18.305714,2.40.70.10,2.40.70.10,214.0,214.0,...,?,N,N,Protein-Protein,Homoprotein,Homodomain (Self),Homodomain,level_4,Homodomain,Intrapolymer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222791,NaN,NaN,424.0,NaN,41.920000,NaN,2.40.70.10,NaN,383.0,NaN,...,NaN,N,NaN,NaN,NaN,NaN,Domain-NaN,NaN,NaN,Interpolymer
2222792,NaN,NaN,120.0,NaN,62.100000,NaN,2.40.70.10,NaN,147.0,NaN,...,NaN,N,NaN,NaN,NaN,NaN,Domain-NaN,NaN,NaN,Interpolymer
2222793,NaN,NaN,144.0,NaN,56.217778,NaN,2.40.70.10,NaN,147.0,NaN,...,NaN,N,NaN,NaN,NaN,NaN,Domain-NaN,NaN,NaN,Interpolymer
2222794,NaN,NaN,227.0,NaN,43.244286,NaN,2.40.70.10,NaN,147.0,NaN,...,NaN,N,NaN,NaN,NaN,NaN,Domain-NaN,NaN,NaN,Interpolymer


In [9]:
df_allsp.to_feather("/homes/2414054/Honours/231006a-AM-honours-project-aps/varalign-data-PF00026/pickles/gz.feather")

ImportError: Missing optional dependency 'pyarrow'.  Use pip or conda to install pyarrow.

In [ ]:
def get_struc_info(df):
    n_res = len(df.drop_duplicates(['UniProt_dbAccessionId_A', 'PDB_dbAccessionId_A', 'PDB_dbChainId_A', 'PDB_dbResNum_A']))
    n_res_un = len(df.drop_duplicates(["SOURCE_ID_A", "Alignment_column_A"]))
    n_strucs =  len(df.PDB_dbAccessionId_A.unique().tolist())
    n_prots = len(df.UniProt_dbAccessionId_A.unique().tolist())
    n_reps = len(df.SOURCE_ID_A.unique().tolist())
    print("The dataframe contains information of:\n{} residues, {} of which are unique\n{} PDB structures\n{} different proteins\n{} unique repeats".format(n_res, n_res_un, n_strucs, n_prots, n_reps))
    
get_struc_info(df_allsp) # UNFILTERED STRUCTURAL DATASET STATS


############## Need to look into why it only 7 proteins. ########################

In [ ]:
df_allsp.UniProt_dbAccessionId_B.value_counts()

In [ ]:
df_allsp.UniProt_dbAccessionId_A.value_counts()

In [ ]:
print(df_allsp.columns.to_numpy())#.tolist())

In [ ]:
df_allsp_rsrz_filt = df_allsp
df_allsp_rsrz_filt.SS_A = df_allsp_rsrz_filt.SS_A.fillna("") # FILL NA WITH "" SO WE CAN PROCESS THE SECONDARY STRUCTURE DATA. NA ARE FOUND ON THOSE RESIDUES PRESENTING NO SS, i.e., COIL
print(df_allsp_rsrz_filt.SS_A)



In [ ]:
get_struc_info(df_allsp_rsrz_filt) # FILTERED STRUCTURAL DATASET STATS

In [ ]:
import Bio.SeqIO
import Bio.AlignIO
from Bio.PDB import *
def get_n_seq(seq_file, seq_format = "fasta"):
    """
    """
    seqs = Bio.SeqIO.parse(seq_file,seq_format)
    n = 0
    for seq in seqs:
        n += 1
    return n

def get_occ_cols(aln_in, fmt_in): #returns a dictionary of the occupoancy of every column in the alignment
    aln = Bio.SeqIO.parse(aln_in, fmt_in)
    occ = {}
    for rec in aln:
        seq = str(rec.seq)
        for i in range(0, len(seq)):
            if i + 1 not in occ:
                occ[i+1] = 0
            if seq[i] != '-':
                occ[i+1] += 1
    return occ

def get_cons_cols(aln_in, fmt_in, t = 0.50): #returns the columns with a relative occupancy greater than a threshold
    """
    """
    n_seq = get_n_seq(aln_in, fmt_in)
    print(n_seq)
    aln_occ = get_occ_cols(aln_in, fmt_in)
    cons_cols = [col for col, occ in aln_occ.items() if occ/n_seq > t]
    return cons_cols
cons_cols_allsp = get_cons_cols('/homes/2414054/varalign_folder/Altered_Alignment/Changing_Alignment/Alignment127_PFAM.sto', 'stockholm') # GETS CONSENSUS COLUMNS FROM MSA (OCCUPANCY > 0.5)
#cons_cols_allsp = get_cons_cols(r'C:\Users\Adam Martin\OneDrive - University of Dundee\Honours Project\Jalview and Chimera using the alignment from PF00026 first from linux\Uniprot_Sequences_For_Alignment_To_Allow_VarAlign_TO\Uniprot_Annotated_Alignment127_PFAM.sto', 'stockholm') # GETS CONSENSUS COLUMNS FROM MSA (OCCUPANCY > 0.5)
print(len(cons_cols_allsp))


In [ ]:
def get_rsa_class_consensus(df, aln_cols):
    """
    """
    df_unique = df.drop_duplicates(['UniProt_dbAccessionId_A', 'PDB_dbAccessionId_A', 'PDB_dbChainId_A', 'PDB_dbResNum_A'])
    df_unique = df_unique.dropna(subset = ["RSA_CLASS_A", "RSA_A"])             #"[RSA_CLASS_UNB_A", "RSA_UNB_A"]   changed to      "[RSA_CLASS_A", "RSA_A"]
    df_unique = df_unique.query('RSA_A != ""')
    df_unique.Alignment_column_A = df_unique.Alignment_column_A.astype(float).astype(int)
    df_cons_cols = df_unique[df_unique.Alignment_column_A.isin(aln_cols)]
    get_struc_info(df_cons_cols)
    grouped_col_rep = df_cons_cols.groupby(["Alignment_column_A", "SOURCE_ID_A"])
    rsa_class_dict = {}
    rsa_dict = {}
    i = 1
    for k, v in grouped_col_rep:
        col = k[0]
        if col not in rsa_class_dict:
            print("Processing consensus column {}".format(i))
            i += 1
            rsa_class_dict[col] = []
        if col not in rsa_dict:
            rsa_dict[col] = []
        
        #holder = []                                     #
        #for x in v.RSA_A.tolist():                      #Changed the RSA_A so that gaps were filled with zeros not sure if this is correct will check later
        #    if x =="":                                  #
        #        x = 0                                   #
        #    holder.append(x)                            #

        rsa_list = list(map(float, v.RSA_A.tolist()))             #v.RSA_A.tolist()))           #RSA_UNB_A
        #print(rsa_list)
        rsa_class_list = v.RSA_CLASS_A.tolist()                 #RSA_CLASS_UNB_A
        try:
            rsa_class_cons = statistics.mode(rsa_class_list)
        except:
            rsa_class_cons = rsa_class_list[0]
        rsa_median = statistics.median(rsa_list)
        rsa_class_dict[col].append(rsa_class_cons)
        rsa_dict[col].append(rsa_median)
    return rsa_class_dict, rsa_dict


In [ ]:
df_allsp.query('Alignment_column_A==565').drop_duplicates(["PDB_dbAccessionId_A","PDB_dbChainId_A"]).RSA_A.to_list()

In [ ]:


def get_rsa_class_consensus(df, aln_cols):
    """
    """
    df_unique = df.drop_duplicates(['UniProt_dbAccessionId_A', 'PDB_dbAccessionId_A', 'PDB_dbChainId_A', 'PDB_dbResNum_A'])
    df_unique = df_unique.dropna(subset = ["RSA_CLASS_A", "RSA_A"])
    df_unique = df_unique.query('RSA_A != ""')
    df_unique.Alignment_column_A = df_unique.Alignment_column_A.astype(float).astype(int)
    df_cons_cols = df_unique[df_unique.Alignment_column_A.isin(aln_cols)]
    missing = []
    for x in aln_cols:
        if x in df_cons_cols['Alignment_column_A'].values:
            t = 0
        else: 
            missing.append(x)
    
    grouped_col_rep = df_cons_cols.groupby(["Alignment_column_A", "SOURCE_ID_A"])
    
    rsa_class_dict = {}
    rsa_dict = {}
    i = 1
    for k, v in grouped_col_rep:
        col = k[0]
        if col not in rsa_class_dict:
            i += 1
            rsa_class_dict[col] = []
        if col not in rsa_dict:
            rsa_dict[col] = []
        for x in missing:
            rsa_dict[x] = 0
            rsa_class_dict[x] = []
    
        

        rsa_list = v.RSA_A.tolist()
        rsa_class_list = v.RSA_CLASS_A.tolist()
        try:
            rsa_class_cons = statistics.mode(rsa_class_list)
        except:
            rsa_class_cons = rsa_class_list[0]
        rsa_median = statistics.median(rsa_list)
        rsa_class_dict[col].append(rsa_class_cons)
        rsa_dict[col].append(rsa_median)

    rsa_dict = dict(sorted(rsa_dict.items()))
    
    rsa_class_dict = dict(sorted(rsa_class_dict.items()))
    
    
    return rsa_class_dict, rsa_dict



                                                                    #######  rsa_list = list(map(float, v.RSA_A.tolist()))       #RSA_UNB_A

rsa_class_dict, rsa_dict = get_rsa_class_consensus(df_allsp_rsrz_filt, cons_cols_allsp) # RETRIEVES RSA DATA FROM TABLE
print(len(rsa_class_dict))
print(rsa_class_dict)

print(len(rsa_dict))
print(rsa_dict)

In [ ]:
def get_rsa_class_df(rsa_class_dict, cons_cols_allsp):
    n = len(rsa_class_dict.keys())
    core = []
    surf = []
    part = []
    for v in rsa_class_dict.values():
        core.append(v.count("Core"))
        surf.append(v.count("Surface"))
        part.append(v.count("Part. Exposed")) 
    df_dssp = pd.DataFrame(list(zip(core, surf, part)), columns =  ["core", "surf", "part"])
    df_dssp["tot"] = df_dssp.core + df_dssp.surf + df_dssp.part
    df_dssp["p_core"] = df_dssp.core / df_dssp.tot
    df_dssp["p_surf"] = df_dssp.surf / df_dssp.tot
    df_dssp["p_part"] = df_dssp.part / df_dssp.tot
    df_dssp["p_tot"] = df_dssp.p_core + df_dssp.p_surf + df_dssp.p_part
    df_dssp.index = range(1, n + 1)         #cons_cols_allsp
    return df_dssp


rsa_class_df = get_rsa_class_df(rsa_class_dict, cons_cols_allsp) # CALCULATES PROPORTION OF RSA CLASS PER POSITION
rsa_class_df

In [ ]:
def plot_rsa_consensus(df_dssp, palette = ["purple", "orange", "royalblue"], bwidth = 1, out = None):                                # bwidth = 1
    core = list(df_dssp.p_core)
    surf = list(df_dssp.p_surf)
    part = list(df_dssp.p_part)
    n_cols = len(df_dssp)
    
    print("n_cols", n_cols)
    cols = list(df_dssp.index)
                 #Added


    
    bottom = []
    for i in range(0, len(core)):
        bottom.append(core[i] + part[i])

    r = (list(np.arange(0, n_cols, 1)))  
    plt.figure(figsize=(180,80))
    plt.rcParams.update({"axes.linewidth": 10})
    plt.bar(r, core,  color=palette[0], edgecolor='black', linewidth = 7.5, width=bwidth, label = 'Core')
    plt.bar(r, part, color=palette[1], bottom = core, edgecolor='black', linewidth = 7.5, width=bwidth, label = 'Part')
    plt.bar(r, surf,  bottom = bottom, color=palette[2], edgecolor='black', linewidth = 7.5, width=bwidth, label = 'Surf')
    plt.xlabel('Occupancy residue position', labelpad = 50, fontsize = 120)
    plt.ylabel('p', labelpad = 50, fontsize = 120)
    legend = plt.legend(loc='center left', bbox_to_anchor=(1, 0.5),fontsize = 120)
    legend.get_frame().set_linewidth(10)
    legend.get_frame().set_edgecolor("black")
    for legobj in legend.legendHandles:
        legobj.set_linewidth(7.5)
    
    plt.tick_params(axis= 'both' , which = 'major', pad = 30, width = 15, length = 50, labelsize = 80)
    
    plt.xticks(np.arange(0, n_cols, 20))     
    plt.yticks(np.arange(0, 1.1, 0.1))
    plt.axhline(y=0.5, linewidth = 5, linestyle = "--", color = 'black')
    plt.xlim(cols[0], cols[-1])                                                           
    plt.ylim(0, 1.025)
    if out != None:
        plt.savefig(out)
    plt.show()

plot_rsa_consensus(rsa_class_df, palette = ["purple", "orange", "royalblue"], bwidth = 1)                                

In [ ]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import seaborn as sns


def medianvs(v):
    if v == 0:
        vs= 0
    else:
        vs = (statistics.median(v))                                     #vs = statistics.median(v) for v in rsa_dict.values()]  #median
    return vs

def medianCI(data, ci, p):
    '''
    data: pandas datafame/series or numpy array
    ci: confidence level
    p: percentile' percent, for median it is 0.5
    output: a list with two elements, [lowerBound, upperBound]
    '''
    if type(data) is pd.Series or type(data) is pd.DataFrame:
        #transfer data into np.array
        data = data.values
    data = data.reshape(-1)
    data = np.sort(data)
    N = data.shape[0]
    lowCount, upCount = stats.binom.interval(ci, N, p, loc=0)
    #given this: https://onlinecourses.science.psu.edu/stat414/node/316
    #lowCount and upCount both refers to  W's value, W follows binomial Dis.
    #lowCount need to change to lowCount-1, upCount no need to change in python indexing
  
    #lowCount -= 1
    upCount -=1
    
   
    return data[int(lowCount)], data[int(upCount)]

out = None

n = len(rsa_dict.keys())
ks = list(range(1, n + 1))                              #index


vs = []
for v in rsa_dict.values():
    vs.append(medianvs(v))
print(rsa_dict)
palettes = {}
c = 1
for v in vs:
    if v <= 5:
        palettes[c] = "Core"
    elif v > 25:
        palettes[c] = "Part"
    else:
        palettes[c] = "Surf"
    c +=1
    
legend_dict = { 'Core' : 'purple', 'Part' : 'orange', 'Surf' : 'royalblue' }

patchList = []
for key in legend_dict:
    data_key = mpatches.Patch(facecolor=legend_dict[key], label=key, edgecolor = "black", linewidth = 3)
    patchList.append(data_key)    
i = 1
lowerbound = []
upperbound = []


medianvs(rsa_dict)
med = medianvs(rsa_dict)
for v in rsa_dict.values():
    
    med = medianvs(v)
    data = pd.Series(v)
    low, up = medianCI(data, 0.95, 0.5)
    lowerbound.append(med-low)
    upperbound.append(up-med)
    if v == 0:
        print(i,v)
    i = i+1

cols = ['vs', 'palettes', 'lowerbound', 'upperbound' ]
lst = []
for a in ks:
    
    lst.append([vs[a-1], palettes[a], lowerbound[a-1], upperbound[a-1], ])
df1 = pd.DataFrame(lst, columns=cols)

hue_order=["purple", "royalblue", "orange"]

ax=sns.barplot(df1, y=df1.vs, x=df1.index, linewidth = 0.01,  hue =df1.palettes, palette=["purple", "royalblue", "orange"]) #, ci=[df1.lowerbound, df1.upperbound]) #, errcolor = "black", errwidth = 7.5, capsize = 35 )
plt.errorbar([k-1 for k in ks], vs, yerr=[lowerbound, upperbound], c = "grey", linewidth = 1, linestyle="None", capsize = 1, capthick = 0.5)
sns.set(rc={'figure.figsize':(50,50)})
#ax.set_title('Median RSA per occupancy position', pad =10, fontsize = 40)
ax.tick_params(axis= 'both' , which = 'major', labelsize = 30) 
ax.set_xlabel("Occupancy residue position", labelpad = 10, fontsize = 60)
ax.set_ylabel('Median RSA', labelpad = 10, fontsize = 60)
ax.axhline(25, linewidth = 2, color = 'black', linestyle = '--')
ax.axhline(5, linewidth = 2, color = 'black', linestyle = '--')





plt.yticks(np.arange(0, 105, 5))
plt.xticks(np.arange(0, n, 10))
plt.ylim(0, 100)  
ax.margins(x=0)
legend = plt.legend(handles=patchList, loc='center left', bbox_to_anchor=(1, 0.5),fontsize = 40)
legend.get_frame().set_linewidth(2)
legend.get_frame().set_edgecolor("black")
plt.figure(figsize=(180,80))
for legobj in legend.legendHandles:
    legobj.set_linewidth(1)
if out != None:
    plt.savefig(out)
plt.show()



In [ ]:
def get_ss_class_consensus(df, aln_cols):
    """
    """
    df_unique = df.drop_duplicates(['UniProt_dbAccessionId_A', 'PDB_dbAccessionId_A', 'PDB_dbChainId_A', 'PDB_dbResNum_A'])
    df_unique = df_unique.dropna(subset = ["SS_CLASS_A"])
    df_unique.Alignment_column_A = df_unique.Alignment_column_A.astype(float).astype(int)
    df_cons_cols = df_unique[df_unique.Alignment_column_A.isin(aln_cols)]
    missing = []
    for x in aln_cols:
        if x in df_cons_cols['Alignment_column_A'].values:
            t = 0
        else: 
            missing.append(x)


    #get_struc_info(df_cons_cols)
    grouped_col_rep = df_cons_cols.groupby(["Alignment_column_A", "SOURCE_ID_A"])
    ss_class_dict = {}
    i = 1
    for k, v in grouped_col_rep:
        col = k[0]
        if col not in ss_class_dict:
            #print("Processing consensus column {}".format(i))
            i += 1
            ss_class_dict[col] = []
        
        for x in missing:
            ss_class_dict[x] = []
            
        
        ss_class_list = v.SS_CLASS_A.tolist()

        try:
            ss_class_cons = statistics.mode(ss_class_list)
        except:
            ss_class_cons = ss_class_list[0]
        ss_class_dict[col].append(ss_class_cons)
        ss_class_dict = dict(sorted(ss_class_dict.items()))
    return ss_class_dict
ss_class_dict = get_ss_class_consensus(df_allsp_rsrz_filt, cons_cols_allsp) # EXTRACTS SS DATA FROM TABLE AND RETURNS DICTIONARY WITH OCCURRENCE OF SS CLASS PER POSTITION
print(len(ss_class_dict))
print(ss_class_dict)

In [ ]:
def get_ss_class_df(rsa_class_dict):
    n = len(rsa_class_dict.keys())
    h = []
    c = []
    e = []
    for v in rsa_class_dict.values():
        h.append(v.count("H"))
        c.append(v.count("C"))
        e.append(v.count("E")) 
    df_dssp = pd.DataFrame(list(zip(h, c, e)), columns =  ["helix", "coil", "strand"])
    df_dssp["tot"] = df_dssp.helix + df_dssp.coil + df_dssp.strand
    df_dssp["p_helix"] = df_dssp.helix / df_dssp.tot
    df_dssp["p_coil"] = df_dssp.coil / df_dssp.tot
    df_dssp["p_strand"] = df_dssp.strand / df_dssp.tot
    df_dssp["p_tot"] = df_dssp.p_helix + df_dssp.p_coil + df_dssp.p_strand
    df_dssp.index = range(1, n + 1)
    return df_dssp
ss_class_df = get_ss_class_df(ss_class_dict) # CALCULATES PROPORTION OF EACH SECONDARY STRUCTURE CLASS PER POISITION
print(len(ss_class_df))
print(ss_class_df)
ss_class_df.loc[98]



In [ ]:
def plot_ss_class_consensus(df_dssp, palette = ["purple", "sienna", "cornflowerblue"], bwidth = 1.2, out = None):
    helix = list(df_dssp.p_helix)
    coil = list(df_dssp.p_coil)
    strand = list(df_dssp.p_strand)

    n_cols = len(df_dssp)
    print(n_cols)
    bottom = []
    for i in range(0, len(helix)):
        bottom.append(strand[i]+helix[i])

    r = list(np.arange(0,n_cols,1))
    plt.figure(figsize=(180,80))
    plt.rcParams.update({"axes.linewidth": 10})
    plt.bar(r, strand,  color=palette[0], edgecolor='black', linewidth = 7.5, width=bwidth, label = 'Strand')
    plt.bar(r, helix, color=palette[1], bottom = strand, edgecolor='black', linewidth = 7.5, width=bwidth, label = 'Helix')
    plt.bar(r, coil,  bottom = bottom, color=palette[2], edgecolor='black', linewidth = 7.5, width=bwidth, label = 'Loop')
    plt.xlabel('Occupancy residue position', labelpad = 50, fontsize = 120)
    plt.ylabel('p', labelpad = 50, fontsize = 120)
    legend = plt.legend(loc='center left', bbox_to_anchor=(1, 0.5),fontsize = 120)
    legend.get_frame().set_linewidth(10)
    legend.get_frame().set_edgecolor("black")
    for legobj in legend.legendHandles:
        legobj.set_linewidth(7.5)
    plt.tick_params(axis= 'both' , which = 'major', pad = 30, width = 15, length = 50, labelsize = 80)
    plt.axhline(y=0.5, linewidth = 5, linestyle = "--", color = 'black')
    plt.yticks(np.arange(0,1.1,0.1))
    plt.xticks(np.arange(0, n_cols, 20))
    plt.xlim(0, n_cols) 
    plt.ylim(0, 1.025)
    if out != None:
        plt.savefig(out)
    plt.show()


plot_ss_class_consensus(ss_class_df, palette = ["purple", "sienna", "cornflowerblue"])

In [ ]:
def get_ss_consensus(df, aln_cols):
    """
    """
    df_unique = df.drop_duplicates(['UniProt_dbAccessionId_A', 'PDB_dbAccessionId_A', 'PDB_dbChainId_A', 'PDB_dbResNum_A'])
    df_unique = df_unique.dropna(subset = ["SS_A"])
    df_unique.Alignment_column_A = df_unique.Alignment_column_A.astype(float).astype(int)
    df_cons_cols = df_unique[df_unique.Alignment_column_A.isin(aln_cols)]

    missing = []
    for x in aln_cols:
        if x in df_cons_cols['Alignment_column_A'].values:
            t = 0
        else: 
            missing.append(x)



    #get_struc_info(df_cons_cols)
    grouped_col_rep = df_cons_cols.groupby(["Alignment_column_A", "SOURCE_ID_A"])
    ss_class_dict = {}
    i = 1
    for k, v in grouped_col_rep:
        col = k[0]
        if col not in ss_class_dict:
            #print("Processing consensus column {}".format(i))
            i += 1
            ss_class_dict[col] = []

        for x in missing:
            ss_class_dict[x] = []

        ss_class_list = v.SS_A.tolist()
        try:
            ss_class_cons = statistics.mode(ss_class_list)
        except:
            ss_class_cons = ss_class_list[0]
        ss_class_dict[col].append(ss_class_cons)
        ss_class_dict = dict(sorted(ss_class_dict.items()))
    return ss_class_dict

ss_dict = get_ss_consensus(df_allsp_rsrz_filt, cons_cols_allsp) # EXTRACTS SECONDARY STRUCTURE DATA FROM TABLE AND RETURNS DICTIONARY WITH OCCURRENCE OF SS ELEMENTS PER POSTITION
print(ss_dict)
print(len(ss_dict))

In [ ]:
def get_ss_df(ss_class_dict):
    n = len(ss_class_dict.keys())
    h = []
    b = []
    e = []
    g = []
    i = []
    t = []
    s = []
    c = []
    for v in ss_class_dict.values():
        h.append(v.count("H"))
        b.append(v.count("B"))
        e.append(v.count("E"))
        g.append(v.count("G"))
        i.append(v.count("I"))
        t.append(v.count("T")) 
        s.append(v.count("S"))
        c.append(v.count(""))
    df_dssp = pd.DataFrame(list(zip(h,b,e,g,i,t,s,c)), columns =  ["a_helix","b_bridge","strand","helix_3_10", "pi_helix","turn","bend","coil"])
    df_dssp["tot"] = df_dssp.a_helix + df_dssp.b_bridge + df_dssp.strand+df_dssp.helix_3_10 + df_dssp.pi_helix + df_dssp.turn+ df_dssp.bend + df_dssp.coil
    df_dssp["p_a_helix"] = df_dssp.a_helix / df_dssp.tot
    df_dssp["p_b_bridge"] = df_dssp.b_bridge / df_dssp.tot
    df_dssp["p_strand"] = df_dssp.strand / df_dssp.tot
    df_dssp["p_3_10_helix"] = df_dssp.helix_3_10 / df_dssp.tot
    df_dssp["p_pi_helix"] = df_dssp.pi_helix / df_dssp.tot
    df_dssp["p_turn"] = df_dssp.turn / df_dssp.tot
    df_dssp["p_bend"] = df_dssp.bend / df_dssp.tot
    df_dssp["p_coil"] = df_dssp.coil / df_dssp.tot
    df_dssp["p_tot"] = df_dssp.p_a_helix + df_dssp.p_b_bridge + df_dssp.p_strand+df_dssp.p_3_10_helix + df_dssp.p_pi_helix +df_dssp.p_turn + df_dssp.p_bend + df_dssp.p_coil
    df_dssp.index = range(1, n + 1)
    return df_dssp

ss_df = get_ss_df(ss_dict) # CALCULATES PROPORTION OF EACH SECONDARY STRUCTURE ELEMENT PER POISITION
print(len(ss_df))
print(ss_df)

In [ ]:
def plot_ss_consensus(df_dssp, palette = ["purple", "darkviolet", "violet", "sienna","sandybrown","grey","blue","cornflowerblue"], bwidth = 1.2, out = None):
    a_helix = list(df_dssp.p_a_helix)
    b_bridge = list(df_dssp.p_b_bridge)
    strand = list(df_dssp.p_strand)
    helix_3_10 = list(df_dssp.p_3_10_helix)
    pi_helix = list(df_dssp.p_pi_helix)
    turn = list(df_dssp.p_turn)
    bend = list(df_dssp.p_bend)
    coil = list(df_dssp.p_coil)
    n_cols = len(df_dssp)
    bottom = [0 for i in range(0, n_cols)]
    #print(bottom)
    #print(len(bottom))
    r = list(np.arange(0,n_cols,1))
    plt.figure(figsize=(180,80))
    plt.rcParams.update({"axes.linewidth": 10})
    plt.bar(r, a_helix,  color=palette[0], bottom = bottom, edgecolor='black', linewidth = 7.5, width=bwidth, label = r'$\alpha$-helix')
    for i in range(0, n_cols):
        bottom[i] += a_helix[i]
    plt.bar(r, helix_3_10, color=palette[1], bottom = bottom,  edgecolor='black', linewidth = 7.5, width=bwidth, label = r'$3_{10}$-helix')
    for i in range(0, n_cols):
        bottom[i] += helix_3_10[i]
    plt.bar(r, pi_helix,  color=palette[2], bottom = bottom,  edgecolor='black', linewidth = 7.5, width=bwidth, label = r'$\pi$-helix')
    for i in range(0, n_cols):
        bottom[i] += pi_helix[i]
    plt.bar(r, b_bridge,  color=palette[3], bottom = bottom, edgecolor='black', linewidth = 7.5, width=bwidth, label = r'$\beta$-bridge')
    for i in range(0, n_cols):
        bottom[i] += b_bridge[i]
    plt.bar(r, strand, color=palette[4], bottom = bottom, edgecolor='black', linewidth = 7.5, width=bwidth, label = r'$\beta$-strand')
    for i in range(0, n_cols):
        bottom[i] += strand[i]
    plt.bar(r, turn,  color=palette[5], bottom = bottom, edgecolor='black', linewidth = 7.5, width=bwidth, label = 'Turn')
    for i in range(0, n_cols):
        bottom[i] += turn[i]
    plt.bar(r, bend,  color=palette[6], bottom = bottom,edgecolor='black', linewidth = 7.5, width=bwidth, label = 'Bend')
    for i in range(0, n_cols):
        bottom[i] += bend[i]
    plt.bar(r, coil, color=palette[7], bottom = bottom, edgecolor='black', linewidth = 7.5, width=bwidth, label = 'Coil')
    plt.xlabel('Domain position', labelpad = 50, fontsize = 120)
    plt.ylabel('p', labelpad = 50, fontsize = 120)
    legend = plt.legend(loc='center left', bbox_to_anchor=(1, 0.5),fontsize = 120)
    #legend = ax.legend()
    legend.get_frame().set_linewidth(10)
    legend.get_frame().set_edgecolor("black")
    for legobj in legend.legendHandles:
        legobj.set_linewidth(7.5)
    plt.tick_params(axis= 'both' , which = 'major', pad = 30, width = 15, length = 50, labelsize = 80)
    plt.axhline(y=0.5, linewidth = 7.5, linestyle = "--", color = 'black')
    plt.yticks(np.arange(0,1.1,0.1))
    plt.xticks(np.arange(0, n_cols, 20))
    plt.xlim(0, n_cols) 
    plt.ylim(0, 1.025)
    if out != None:
        plt.savefig(out)
    plt.show()
plot_ss_consensus(ss_df)

In [ ]:
ss_df

## CONTACT MAPS

In [ ]:
import pandas as pd
#import structural_analysis
import statistics
#import structure_validation
import importlib
from scipy import stats
import math
import numpy as np
from pathlib import Path  
import vcf
import matplotlib.patches as mpatches


import matplotlib.pyplot as plt
import seaborn as sns




In [ ]:
#df_allsp = pd.read_pickle(r"C:\Users\Adam Martin\ONEDRI~1\HONOUR~1\JALVIE~1\UNIPRO~1\UNIPRO~1.GZ")
df_allsp = pd.read_pickle(r'/homes/2414054/varalign_folder/Altered_Alignment/Changing_Alignment/Uniprot_Annotated_Alignment127_PFAM.sto_prointvar_structure_table.p.gz')


In [ ]:
df_allsp.shape
df_allsp

In [ ]:
aln_in = "/homes/2414054/varalign_folder/Altered_Alignment/Changing_Alignment/Alignment127_PFAM.sto" # MSA
#aln_in = r"C:\Users\Adam Martin\OneDrive - University of Dundee\Honours Project\Jalview and Chimera using the alignment from PF00026 first from linux\Uniprot_Sequences_For_Alignment_To_Allow_VarAlign_TO\Uniprot_Annotated_Alignment127_PFAM.sto"
  
aln_fmt = "stockholm" # MSA FORMAT
    

In [ ]:

import Bio.SeqIO
import Bio.AlignIO
import statistics
import importlib
from Bio.PDB import *

def get_n_seq(seq_file, seq_format = "fasta"):
    """
    """
    seqs = Bio.SeqIO.parse(seq_file,seq_format)
    n = 0
    for seq in seqs:
        n += 1
    return n

def get_occ_cols(aln_in, fmt_in): #returns a dictionary of the occupoancy of every column in the alignment
    aln = Bio.SeqIO.parse(aln_in, fmt_in)
    occ = {}
    for rec in aln:
        seq = str(rec.seq)
        for i in range(0, len(seq)):
            if i + 1 not in occ:
                occ[i+1] = 0
            if seq[i] != '-':
                occ[i+1] += 1
    return occ

def get_cons_cols(aln_in, fmt_in, t = 0.5): #returns the columns with a relative occupancy greater than a threshold
    """
    """
    n_seq = get_n_seq(aln_in, fmt_in)
    aln_occ = get_occ_cols(aln_in, fmt_in)
    cons_cols = [col for col, occ in aln_occ.items() if occ/n_seq > t]
    return cons_cols


cons_cols_allsp = get_cons_cols(aln_in, aln_fmt) # GETS CONSENSUS COLUMNS FROM MSA (OCCUPANCY > 0.5)
print(len(cons_cols_allsp))



########################################################################################################################################################

In [ ]:
def get_cons_cols_eq(cons_cols):
    cons_cols_eq = {}
    for i, col in enumerate(cons_cols):
        cons_cols_eq[col] = i+1
    return cons_cols_eq

In [ ]:
def format_df(df, cons_cols):
	cons_cols_eq = get_cons_cols_eq(cons_cols)
	df2 = df.copy(deep = True)
	df2_filt = df2.dropna(subset = ["Alignment_column_A", "Alignment_column_B"])
	df2_filt.Alignment_column_A = df2_filt.Alignment_column_A.astype(int)
	df2_filt.Alignment_column_B = df2_filt.Alignment_column_B.astype(int)
	df2_filt.UniProt_dbResNum_A = df2_filt.UniProt_dbResNum_A.astype(int)
	df2_filt.UniProt_dbResNum_B = df2_filt.UniProt_dbResNum_B.astype(int)
	df2_cons = df2_filt[(df2_filt.Alignment_column_A.isin(cons_cols)) & (df2_filt.Alignment_column_B.isin(cons_cols))]
	df2_cons["Alignment_column_cons_A"] = df2_cons.Alignment_column_A.map(cons_cols_eq)
	df2_cons["Alignment_column_cons_B"] = df2_cons.Alignment_column_B.map(cons_cols_eq)
	return df2_cons


In [ ]:
df_rf = format_df(df_allsp, cons_cols_allsp) # FORMATS STRUCTURAL TABLE FOR FOLLOW-UP ANALYSIS
df_rf			                    

In [ ]:
def get_df_cols_eq(df):
    cols = df.columns.tolist()
    un_cols = list(set([col[:-2] for col in cols]))
    cols_eq = {}
    for col in cols:
        if col[-2:] == "_A":
            cols_eq[col] = col[:-2] + "_B"
        elif col[-2:] == "_B":
            cols_eq[col] = col[:-2] + "_A"
        else:
            cols_eq[col] = col
    return cols_eq

cons_cols_eq = get_cons_cols_eq(cons_cols_allsp) # EQUIVALENCE BETWEEN CONSENSUS ALIGNMENT COLUMN NUMBERS AND DOMAIN POSITIONS (1-33)
print(cons_cols_eq)
df_cols_eq = get_df_cols_eq(df_rf)
print(df_cols_eq)

In [ ]:
#def get_res_in_rep(df, rep_id):
#    df_rep = df[(df.SOURCE_ID_A == rep_id)&(df.SOURCE_ID_B == rep_id)]
#    res_a = df_rep.Alignment_column_A.unique().tolist()                         #maybe this should be concensus instead of alignment column
#    res_b = df_rep.Alignment_column_B.unique().tolist()
#    res = sorted(list(set(res_a + res_b)))
#    return res

def get_res_in_rep(df, rep_id, cons_eq):
    df_rep = df[(df.SOURCE_ID_A == rep_id)&(df.SOURCE_ID_B == rep_id)]
    res_a = df_rep.Alignment_column_A.unique().tolist()
    res_b = df_rep.Alignment_column_B.unique().tolist()
    res = sorted(list(set(res_a + res_b)))
    cons_res = [cons_eq[r] for r in res]
    return cons_res



def get_struc_info(df):
    n_res = len(df.drop_duplicates(['UniProt_dbAccessionId_A', 'PDB_dbAccessionId_A', 'PDB_dbChainId_A', 'PDB_dbResNum_A']))
    n_res_un = len(df.drop_duplicates(["SOURCE_ID_A", "Alignment_column_A"]))
    n_strucs =  len(df.PDB_dbAccessionId_A.unique().tolist())
    n_prots = len(df.UniProt_dbAccessionId_A.unique().tolist())
    n_reps = len(df.SOURCE_ID_A.unique().tolist())
    print("The dataframe contains information of:\n{} residues, {} of which are unique\n{} PDB structures\n{} different proteins\n{} unique repeats".format(n_res, n_res_un, n_strucs, n_prots, n_reps))
    

#def get_intra_cons_occ(df, cons_cols_eq):
#    reps = df.SOURCE_ID_A.unique().tolist()
#    res_occ = {}
#    idseq = []
#    for rep in reps:
#        rep_res = get_res_in_rep(df, rep)
#        #print(rep_res)
#        for i in rep_res:
#            for j in rep_res:
#                idx = tuple([cons_cols_eq[i], cons_cols_eq[j]])
#                if idx not in res_occ:
#                    res_occ[idx] = 0
#                res_occ[idx] += 1
#                #if i not in idseq: 
#                #    idseq.append(i)
#                #if j not in idseq: 
#                #    idseq.append(j)
#        #idseq.sort()
#    return res_occ, #idseq



def get_intra_cons_occ(df, cons_eq):
    get_struc_info(df)
    reps = df.SOURCE_ID_A.unique().tolist()
    res_occ = {}
    for rep in reps:
        rep_res = get_res_in_rep(df, rep, cons_eq)
        for i in rep_res:
            for j in rep_res:
                idx = tuple([i, j])
                if idx not in res_occ:
                    res_occ[idx] = 0
                res_occ[idx] += 1
    return res_occ



intra_res_occ  = get_intra_cons_occ(df_rf, cons_cols_eq) # GETS COVERAGE OF EACH INTRA-REPEAT POSITION PAIR IN OUR STRUCTURAL DATASET

intra_res_occ

In [ ]:
def symmetric_contact_matrix(df):
    df2 = pd.DataFrame.copy(df, deep = True)
    rows = list(df.index)
    cols = list(df.columns)
    for i in rows:
        for j in cols[i-1:]:
            df2.loc[j,i] = df2.loc[i,j]
    return df2

#def symmetric_contact_matrix(df, cons_cols):
#    df2 = pd.DataFrame.copy(df, deep = True)
#    rows = list(df.index)
#    cols = list(df.columns)
#    x=0
#    for i in rows:
#        x +=1
#        for j in cols[x-1:]:
#            #if  df2.loc[i,j] > 0:
#                #print(i,j, df2.loc[i,j])
#            df2.loc[j,i] = df2.loc[i,j]
#    return df2

#def get_intra_cons(df, cons_cols_eq, t = 0, int_mask = None):
#    real_cols = list(cons_cols_eq.keys())
#    cons_cols = list(cons_cols_eq.values())
#    df2 = df.copy(deep = True)
#    if int_mask != None:
#        df2 = df2[df2.Int_Types.str.contains(int_mask)]
#    contact_matrix_intra = pd.DataFrame(np.nan, index = cons_cols, columns = cons_cols).fillna(0)
#    if t == 0:
#        df_filt = df2[df2.SOURCE_ID_A == df2.SOURCE_ID_B].groupby("SOURCE_ID_A")
#    else:
#        df_filt = df2[(df.SOURCE_ID_A == df2.SOURCE_ID_B) & (abs(df2.Alignment_column_cons_A - df2.Alignment_column_cons_B) > t)].groupby("SOURCE_ID_A")
#    for repeat, row in df_filt:
#        repeat_crosstab = pd.crosstab(row.Alignment_column_cons_A,
#                                      row.Alignment_column_cons_B)    # creates a cross table of how many contacts there are between any two given residues
#        repeat_crosstab = repeat_crosstab.reindex(cons_cols).fillna(0)         # changes index (rows) so it includes all positions (1-33)
#        repeat_crosstab = repeat_crosstab.reindex(columns = cons_cols).fillna(0) # changes column index so it includes all positions (1-33)
#        repeat_crosstab[repeat_crosstab >= 1] = 1 # flattening all interatomic interactions and overrepresented interactions in multiple structure to value of one
#        contact_matrix_intra += repeat_crosstab  # adding each crosstab for each repeat to the contact matrix dataframe to study all interactions
#    contact_matrix_intra_symm = symmetric_contact_matrix(contact_matrix_intra)
#    return contact_matrix_intra_symm

def get_intra_cons(df, cons_cols_eq, t = 0, int_mask = None):
    real_cols = list(cons_cols_eq.keys())
    cons_cols = list(cons_cols_eq.values())
    df2 = df.copy(deep = True)
    if int_mask != None:
        df2 = df2[df2.Int_Types.str.contains(int_mask)]
    contact_matrix_intra = pd.DataFrame(np.nan, index = cons_cols, columns = cons_cols).fillna(0)
    if t == 0:
        df_filt = df2[df2.SOURCE_ID_A == df2.SOURCE_ID_B].groupby("SOURCE_ID_A")
    else:
        df_filt = df2[(df.SOURCE_ID_A == df2.SOURCE_ID_B) & (abs(df2.Alignment_column_cons_A - df2.Alignment_column_cons_B) > t)].groupby("SOURCE_ID_A")
    for repeat, row in df_filt:
        repeat_crosstab = pd.crosstab(row.Alignment_column_cons_A,
                                      row.Alignment_column_cons_B)    # creates a cross table of how many contacts there are between any two given residues
        repeat_crosstab = repeat_crosstab.reindex(cons_cols).fillna(0)         # changes index (rows) so it includes all positions (1-33)
        repeat_crosstab = repeat_crosstab.reindex(columns = cons_cols).fillna(0) # changes column index so it includes all positions (1-33)
        repeat_crosstab[repeat_crosstab >= 1] = 1 # flattening all interatomic interactions and overrepresented interactions in multiple structure to value of one
        contact_matrix_intra += repeat_crosstab  # adding each crosstab for each repeat to the contact matrix dataframe to study all interactions
    print(contact_matrix_intra)
    contact_matrix_intra_symm = symmetric_contact_matrix(contact_matrix_intra)
    return contact_matrix_intra_symm    


intra = get_intra_cons(df_rf, cons_cols_eq, t = 0) # CALCULATES ABSOLUTE FREQUENCY OF INTRA-REPEAT CONTACTS                 #Changed the consensus sequences used to one using just intra make the matrix and graph look better
intra                                                                                                

In [ ]:
#def normalize_contacts(df, res_occ_dict):
#    max = 0
#    max2=0
#    df2 = df.copy(deep = True)
#    df3 = df.copy(deep = True)
#    for k, v in res_occ_dict.items():                                                                       #res_occ_dict is missing the start and end data. 134 is supposed to 7 and 968 is supossed to be 7
#        
#        if k[0] > max:
#            max = k[0]
#            #print("max", max)
#        if k[1] > max2:
#            max2 = k[1]
#            #print("max", max)#
#
#        df3.loc[k[1],k[0]] = df2.loc[k[0],k[1]]/v
#        df3.loc[k[0],k[1]] = 0                            
#        
#    
##   
 #   
 #   print("max", max, max2)
 #   return df3


def normalize_contacts(df, res_occ_dict, Half="False"):
    df2 = df.copy(deep = True)
    df3 = df.copy(deep = True)
    for k, v in res_occ_dict.items():
        
        df3.loc[k[1],k[0]] = df2.loc[k[0],k[1]]/v
        if Half=="True":
            df3.loc[k[0],k[1]] = 0                            
    return df3

intra_norm = normalize_contacts(intra, intra_res_occ, Half="False") # NORMALISES CONTACT MAP BY POSITION PAIR COVERAGE IN DATASET

# Sorted out the normalising bit, seemed to be out of indexing. Now not sure about the 134 bit.

In [ ]:
#def plot_intra_cons(df, df_cols_eq, cmap = 'Greens', out = None):
#    
#    n_cols = len(df)
#    print(n_cols)
#    fig, ax = plt.subplots(1, 1, figsize=(200, 160))
#    ax = sns.heatmap(df, cmap = cmap, square = True, linewidths = 5, linecolor = 'black', cbar_kws = dict(ticks = np.arange(0,1,1)))
#    ax.xaxis.tick_top()
#    ax.yaxis.tick_left()
#    ax.tick_params(axis= 'both' , labelrotation = 'auto', which = 'major', pad = 60, width = 15, length = 50, labelsize = 80)
#    ax.figure.axes[-1].yaxis.label.set_size(180)
#    cbar = ax.collections[0].colorbar
#    cbar.set_label('p', labelpad=190)
#    cbar.ax.tick_params(pad = 60, width = 5, length = 30, labelsize = 180)
##    plt.xlabel("Occupancy residue position", fontsize = 120, labelpad = 100)
#    plt.ylabel("Occupancy residue position", fontsize = 120, labelpad = 100)
##    plt.xticks(np.arange(0, n_cols, 20))
 #   plt.yticks(np.arange(0, n_cols, 20))
 #   ax.xaxis.set_label_position('top')
#    if out != None:
#        plt.savefig(out)
#    plt.show()


def plot_intra_cons(df, cmap = 'Greens', out = None):
    rage = np.arange(0, len(df), 20)
    
    fig, ax = plt.subplots(1, 1, figsize=(200, 160))
    ax = sns.heatmap(df, cmap = cmap, square = True, linewidths = 5, linecolor = 'black', cbar_kws = dict(ticks = np.arange(0,1.1,0.1)))
    ax.xaxis.tick_top()
    ax.yaxis.tick_left()
    ax.set_xticks(np.arange(0, len(df), 20), labels= rage)
    ax.set_yticks(np.arange(0, len(df), 20), labels= rage)
    ax.tick_params(axis= 'both' , labelrotation = 'auto', which = 'major', pad = 60, width = 5, length = 30, labelsize = 180)
    ax.figure.axes[-1].yaxis.label.set_size(180)
    ax.set_ylim(len(df),0)
    ax.set_xlim(0, len(df))
    
    cbar = ax.collections[0].colorbar
    cbar.set_label('p', labelpad=190)
    cbar.ax.tick_params(pad = 60, width = 5, length = 30, labelsize = 180)
    plt.xlabel("Occupancy residue position", fontsize = 180, labelpad = 60)
    plt.ylabel("Occupancy residue position", fontsize = 180, labelpad = 60)
    plt.tick_params( which = 'major', pad = 30, width = 15, length = 50, labelsize = 80)
    
    ax.xaxis.set_label_position('top')
    if out != None:
        plt.savefig(out)
    plt.show()

plot_intra_cons(intra_norm, "viridis")
print(len(intra_norm))

In [ ]:
intra_t5 = get_intra_cons(df_rf, cons_cols_eq, t = 5) # ONLY PAIRS OF POSITIONS FURTHER AWAY THAN 5 POSITIONS FROM EACH OTHER
intra_norm_t5 = normalize_contacts(intra_t5, intra_res_occ) # NORMALISATION

print(intra_norm_t5)

In [ ]:
plot_intra_cons(intra_norm_t5, "viridis")
print(len(intra_norm_t5))

In [ ]:
def get_tot_occ_res(occ_dict, n_cols, contact_map = "intra", t = 0):
    res_occ = {}
    for i in list(occ_dict.keys()):
        if i[0] not in res_occ:
            res_occ[i[0]] = 0
        if contact_map == "intra":
            if abs(i[1] - i[0]) > t:
                res_occ[i[0]] += occ_dict[(i[0],i[0])]
    for x in range(1,n_cols,1):
        Found = "False"
        for y in res_occ.keys():
            if x == y:
                Found = "True"
        if Found == "False":
            res_occ[x] = 0
    res_occ = dict(sorted(res_occ.items()))
    return res_occ


        



def get_tot_cons_res(cons_df, contact_map = "intra", t = 0):
    res_cons = {}
    idx = cons_df.index.tolist()
    for i in idx:
        if i not in res_cons:
            res_cons[i] = 0
        for j in idx:
            if contact_map == "intra":
                
                if abs(j - i) > t:
                    res_cons[i] += cons_df.loc[i,j]
            elif contact_map == "inter":
                if i == j:
                    res_cons[i] += cons_df.loc[i,j]
                else:
                    res_cons[i] += (cons_df.loc[i,j] + cons_df.loc[j,i])         
    return res_cons

def get_OR(df):
    df.contacts = df.contacts + 1 #pseudocount
    tot_occ = sum(df.occ)
    tot_cons = sum(df.contacts)
    for i in df.index:
        i_occ = df.loc[i,"occ"]
        i_cons = df.loc[i,'contacts']

        rest_occ = tot_occ - i_occ
        rest_cons = tot_cons - i_cons
        
        oddsr, pval = stats.fisher_exact([[i_cons, rest_cons], [i_occ, rest_occ]])
        vals = [i_cons,rest_cons,i_occ, rest_occ]
        se_logor = 1.96*(math.sqrt(sum(list(map((lambda x: 1/x),vals)))))
        logor = math.log(oddsr)
        df.loc[i,'oddsratio'] = oddsr
        df.loc[i,'log_oddsratio'] = logor
        df.loc[i,'pvalue'] = pval
        df.loc[i,'ci_dist'] = se_logor
    return df




def get_OR_from_cons(res_cons, res_occ):
    cons = list(res_cons.values())
    occ = list(res_occ.values())
    df = pd.DataFrame(list(zip(occ, cons)), columns =['occ', 'contacts'])
    df.index = range(1,len(df)+1)
    df = get_OR(df)
    return df




def add_colours(df, missense_variants_df ):
    jioner = missense_variants_df[['miss_class','miss_color']].copy()
    df2 = pd.merge(df, jioner, left_index=True, right_index=True)
    return df2


def fisher_exact_test(df,x):
    
    cmd_cons = df[df.miss_class == x].contacts.sum()
    cmd_occ = df[df.miss_class == x].occ.sum()
    rest_cons = df.contacts.sum() - cmd_cons
    rest_occ = df.occ.sum() - cmd_occ
    oddsr1, pval1 = stats.fisher_exact([[cmd_cons, rest_cons], [cmd_occ, rest_occ]]) # CMDs ARE NOT ENRICHED IN INTER-REPEAT CONTACTS

    cmd_cons = df[df.miss_class == x].contacts.sum()
    cmd_occ = df[df.miss_class == x].occ.sum()
    rest_cons = df[df.miss_class != x].contacts.sum()
    rest_occ = df[df.miss_class != x].occ.sum()
    oddsr2, pval2 = stats.fisher_exact([[cmd_cons, rest_cons], [cmd_occ, rest_occ]]) # CMDs ARE ENRICHED IN INTRA-REPEAT CONTACTS RELATIV
    if oddsr1 == oddsr2 and pval1 == pval2:
        print(x, oddsr1, pval1)
    else:
        print(x, oddsr1, pval1)
        print()
        print(x, oddsr2, pval2)
    

def add_miss_class(df, cons_col = "shenkin", thresholds = [30, 70], colours = ["royalblue", "seagreen", "grey", "purple", "orange"]):
    for i in df.index:          
        if df.loc[i, cons_col] <= thresholds[0] and df.loc[i, "log_oddsratio"] < 0:
            df.loc[i,"miss_class"] = "CMD"
        elif df.loc[i, cons_col] <= thresholds[0] and df.loc[i, "log_oddsratio"] > 0:
            df.loc[i,"miss_class"] = "CME"
        elif df.loc[i, cons_col] >= thresholds[1] and df.loc[i, "log_oddsratio"] < 0:
            df.loc[i,"miss_class"] = "UMD"
        elif df.loc[i, cons_col] >= thresholds[1] and df.loc[i, "log_oddsratio"] > 0:
            df.loc[i,"miss_class"] = "UME"
        else:
            df.loc[i,"miss_class"] = "None"
           
    coloring = {
        "CMD": colours[0],
        "CME": colours[1],
        "UMD": colours[3],
        "UME": colours[4],
        "None": colours[2]
    }
    df["miss_color"] =  df.miss_class.map(coloring) 
    return df

In [ ]:
#occ_intra = get_tot_occ_res(intra_res_occ, len(intra), contact_map = "intra", t = 5) # STRUCTURAL COVERAGE PER POSITION
#cons_intra = get_tot_cons_res(intra, contact_map = "intra", t = 5) # TOTAL NUMBER OF CONTACTS PER POSITION
#enrichment_df_intra = get_OR_from_cons(cons_intra, occ_intra) # CALCULATES ENRICHMENT SCORES IN INTRA-REPEAT CONTACTS, P-VALUE, AND 95% CI FOR ENRICHMENT SCORE
#enrichment_df_intra = add_miss_class(enrichment_df_intra) # ADDS MISSENSE ENRICHMENT CLASS

# CONSERVATION AND VARIATION ANALYSIS

In [ ]:
import math
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import Bio.SeqIO
import Bio.AlignIO
import statistics
import importlib
from Bio.PDB import *
import os
import pandas as pd
import matplotlib.patches as mpatches

In [ ]:
variant_table = pd.read_pickle("/homes/2414054/varalign_folder/Altered_Alignment/Changing_Alignment/Uniprot_Annotated_Alignment127_PFAM.sto_variants.p.gz") # VARIANT TABLE
aln_in = "/homes/2414054/varalign_folder/Altered_Alignment/Changing_Alignment/Alignment127_PFAM.sto" # MSA
#aln_in = r"C:\Users\Adam Martin\OneDrive - University of Dundee\Honours Project\Jalview and Chimera using the alignment from PF00026 first from linux\Uniprot_Sequences_For_Alignment_To_Allow_VarAlign_TO\Uniprot_Annotated_Alignment127_PFAM.sto"
  
aln_fmt = "stockholm" # MSA FORMAT
    

In [ ]:
def get_occ_cols(aln_in, fmt_in): #returns a dictionary of the occupoancy of every column in the alignment
    aln = Bio.SeqIO.parse(aln_in, fmt_in)
    occ = {}
    for rec in aln:
        seq = str(rec.seq)
        for i in range(0, len(seq)):
            if i + 1 not in occ:
                occ[i+1] = 0
            if seq[i] != '-':
                occ[i+1] += 1
    return occ


def get_n_seq(seq_file, seq_format = "fasta"):
    """
    """
    seqs = Bio.SeqIO.parse(seq_file,seq_format)
    n = 0
    for seq in seqs:
        n += 1
    return n


def get_cons_cols(aln_in, fmt_in, t = 0.5): #returns the columns with a relative occupancy greater than a threshold
    """
    """
    n_seq = get_n_seq(aln_in, fmt_in)
    aln_occ = get_occ_cols(aln_in, fmt_in)
    cons_cols = [col for col, occ in aln_occ.items() if occ/n_seq > t]
    return cons_cols

cons_cols = get_cons_cols(aln_in, aln_fmt, t = 0.50) # CONTAINS INDICES OF MSA COLUMNS WITH OCCUPANCY > 0.50 OR CONSENSUS
print(len(cons_cols))

In [ ]:
def in_columns(aln_in, infmt):
    aln = Bio.AlignIO.read(aln_in, infmt)
    n_cols = len(aln[0])
    cols = {}
    for col in range(1,n_cols+1):
        cols[col] = []
    for row in aln:
        seq = str(row.seq)
        for i in range(0,len(seq)):
            cols[i+1].append(seq[i])
    return cols
def get_freqs(col):
    abs_freqs = {
        'A':0, 'R':0, 'N':0, 'D':0,'C':0,'Q':0, 'E':0,'G':0,'H':0, 'I':0,
        'L':0, 'K':0,'M':0, 'F':0, 'P':0,'S':0,'T':0, 'W':0, 'Y':0, 'V':0, '-':0
    }
    for aa in col:
        aa = aa.upper()
        if col.count('-') == len(col):
            abs_freqs['-'] = 1
            return abs_freqs
        if aa not in ['-', 'X', 'B']:
            abs_freqs[aa] += 1
    rel_freqs = {k: v/(len(col)-(col.count('-')+col.count('X')+col.count('B'))) for k, v in abs_freqs.items()}
    return rel_freqs
def get_entropy(freqs):
    S = 0
    for f in freqs.values():
        if f != 0:
            S += f*math.log2(f)
    return -S
def get_shenkin(col):
    S = get_entropy(get_freqs(col))
    return (2**S)*6
def get_stats(col):
    n_seqs = len(col)
    gaps = col.count('-')
    occ = n_seqs - gaps
    occ_pct = occ/n_seqs
    gaps_pct = 1 - occ_pct
    return occ, gaps, occ_pct, gaps_pct
def calculate_shenkin(aln_in, infmt, t = 0.5):
    cols = in_columns(aln_in, infmt)
    scores = []
    occ = []
    gaps = []
    occ_pct = []
    gaps_pct = []
    for k, v in cols.items():
        scores.append(get_shenkin(v))
        stats = (get_stats(v))
        occ.append(stats[0])
        gaps.append(stats[1])
        occ_pct.append(stats[2])
        gaps_pct.append(stats[3])
    df = pd.DataFrame(list(zip(list(range(1,len(scores)+1)),scores, occ,gaps, occ_pct, gaps_pct)), columns = ['col','shenkin','occ','gaps','occ_pct','gaps_pct'])
    #df = df[df.occ_pct >= t]
    #max_shenkin = max(list(df.shenkin))
    #df.index = range(1, len(df)+1)
    return df
shenkin_aln = calculate_shenkin(aln_in, aln_fmt)
shenkin_aln

In [ ]:
shenkin_aln_filt = shenkin_aln[shenkin_aln.occ_pct > 0.50]
shenkin_aln_filt.index = range(1, len(shenkin_aln_filt)+1) # CONTAINS SHENKIN SCORE, OCCUPANCY/GAP PROPORTION OF CONSENSUS COLUMNS
shenkin_aln_filt

In [ ]:
min_shenkin = min(shenkin_aln_filt.shenkin)

shenkin_aln_filt.shenkin = shenkin_aln_filt.shenkin.astype(int)
max_shenkin = np.max(shenkin_aln_filt.shenkin)
shenkin_aln_filt["norm_shenkin_rel"] = 100*(shenkin_aln_filt.shenkin-min_shenkin)/(max_shenkin-min_shenkin) # ADDING NEW COLUMNS WITH DIFFERENT NORMALISED SCORES
shenkin_aln_filt["norm_shenkin_abs"] = 100*(shenkin_aln_filt.shenkin-6)/(120-6)
shenkin_aln_filt

In [ ]:
def plot_conservation_mod(df, colors = ["steelblue", "orange", "tomato", "orchid"], cons_score_col = 'shenkin', cons_score_label = "Shenkin score", qs = [60,], out = None):



    n_cols = len(df)
    print(n_cols)

    plt.figure(figsize=(180,80))
    plt.rcParams.update({"axes.linewidth": 10})
    palette = {}
   
    
    
    for i in df.index:
        if df.loc[i, cons_score_col] <= qs[0]:
            palette[i] = colors[0]
        elif df.loc[i, cons_score_col] > qs[0] and df.loc[i, cons_score_col] <= qs[1]:
            palette[i] = colors[1]
        elif df.loc[i, cons_score_col] > qs[1] and df.loc[i, cons_score_col] <= qs[2]:
            palette[i] = colors[2]
        else:
            palette[i] = colors[3]

    
            #r'$3_{10}$-helix'
    legend_dict = { r'$N_{s} \leq 25$' : colors[0], r'$25 < N_{s} \leq 50$' : colors[1], r'$50 < N_{s} \leq 75$' : colors[2], r'$N_{s} > 75$' : colors[3]}
    patchList = []
    for key in legend_dict:
        data_key = mpatches.Patch(facecolor=legend_dict[key], label=key, edgecolor = "black", linewidth = 3)
        patchList.append(data_key)

    cols = ['SHENKIN', 'palettes']
    lst = []
    for x in df.index:
        lst.append([df.norm_shenkin_rel  [x], palette[x]])
    df1 = pd.DataFrame(lst, columns=cols)
    

    
    

    
    ax=sns.barplot(df1, y=df1.SHENKIN, x=df1.index, linewidth = 0.01,  hue =df1.palettes, palette=["purple", "royalblue", "orange", "seagreen"], edgecolor = 'black')
    ax.tick_params(axis= 'both' , which = 'major', pad = 30, width = 15, length = 50, labelsize = 80)
    ax.set_xlabel("Occupancy residue position", labelpad = 50, fontsize = 120)
    ax.set_ylabel(cons_score_label, labelpad = 50, fontsize = 120)
    for i, q in enumerate(qs):
        plt.axhline(y = q, color = "black", linewidth = 7.5, linestyle = '--') #colors[i]
    plt.xlim(0,len(df))
    plt.xticks(np.arange(0, n_cols, 20))
    legend = plt.legend(handles=patchList, loc='center left', bbox_to_anchor=(1, 0.5),fontsize = 120)
    legend.get_frame().set_linewidth(10)
    legend.get_frame().set_edgecolor("black")
    for legobj in legend.legendHandles:
        legobj.set_linewidth(7.5)
    if out != None:
        plt.savefig(out)
    plt.show()



#"steelblue", "forestgreen", "gold", "firebrick"

plot_conservation_mod(shenkin_aln_filt, colors = ["purple", "seagreen", "royalblue", "orange"],
                                       cons_score_col = 'norm_shenkin_rel', cons_score_label = "Normalised Shenkin divergence score", qs = [25, 50, 75]) # PLOT NORMALISED SHENKIN DIVERGENCE SCORE PER POSITION

## MISSENSE VARIANTS ANALYSIS

In [ ]:



############################################################################################WRONG AFTER THIS 


def format_variant_table(df, col_mask, vep_mask = ["missense_variant"], tab_format = "gnomad"):
    df_filt = df.copy(deep = True)
    df_filt.reset_index(inplace=True)
    if tab_format == "gnomad":
        df_filt.columns = [' '.join(col).strip() for col in df_filt.columns.tolist()]
    df_filt.columns = [col.lower().replace(" ", "_") for col in df_filt.columns.tolist()]
    df_filt = df_filt[df_filt.source_id.str.contains("HUMAN")]
    
    df_filt = df_filt.dropna(subset=["vep_consequence"])
    df_filt = df_filt[df_filt.vep_consequence.isin(vep_mask) ]
    df_filt = df_filt[df_filt.alignment_column.isin(col_mask)]
    return df_filt
human_missense_variants = format_variant_table(variant_table, cons_cols) # GET ONLY MISSENSE VARIANTS ROWS
human_missense_variants


def format_variant_table(df, col_mask, vep_mask = ["synonymous_variant"], tab_format = "gnomad"):
    df_filt = df.copy(deep = True)
    df_filt.reset_index(inplace=True)
    if tab_format == "gnomad":
        df_filt.columns = [' '.join(col).strip() for col in df_filt.columns.tolist()]
    df_filt.columns = [col.lower().replace(" ", "_") for col in df_filt.columns.tolist()]
    df_filt = df_filt[df_filt.source_id.str.contains("HUMAN")]
    
    df_filt = df_filt.dropna(subset=["vep_consequence"])
    df_filt = df_filt[df_filt.vep_consequence.isin(vep_mask) ]
    df_filt = df_filt[df_filt.alignment_column.isin(col_mask)]
    return df_filt
human_synomous_variants = format_variant_table(variant_table, cons_cols) # GET ONLY human_synomous_variants VARIANTS ROWS
human_synomous_variants

In [ ]:
def generate_subset_aln(aln_in, aln_fmt, df, aln_out = None):
    seqs_ids = df.source_id.unique().tolist()
    aln = Bio.SeqIO.parse(aln_in, aln_fmt)
    variant_seqs = [rec for rec in aln if rec.id in seqs_ids]
    if aln_out == None:
        pref, fmt = aln_in.split(".")
        aln_out =  pref + "_variant_seqs."+fmt
    Bio.SeqIO.write(variant_seqs, aln_out, aln_fmt)
    return aln_out

def calculate_shenkin(aln_in, infmt, t = 0.5):
    cols = in_columns(aln_in, infmt)
    scores = []
    occ = []
    gaps = []
    occ_pct = []
    gaps_pct = []
    for k, v in cols.items():
        scores.append(get_shenkin(v))
        stats = (get_stats(v))
        occ.append(stats[0])
        gaps.append(stats[1])
        occ_pct.append(stats[2])
        gaps_pct.append(stats[3])
    df = pd.DataFrame(list(zip(list(range(1,len(scores)+1)),scores, occ,gaps, occ_pct, gaps_pct)), columns = ['col','shenkin','occ','gaps','occ_pct','gaps_pct'])
    df = df[df.occ_pct >= t]
    #max_shenkin = max(list(df.shenkin))
    df.index = range(1, len(df)+1)
    return df

def get_OR(df, variant_col = "variants"):
    tot_occ = sum(df.occ)
    tot_vars = sum(df[variant_col])
    idx = df.index.tolist()
    for i in idx:
        i_occ = df.loc[i,"occ"]
        i_vars = df.loc[i,variant_col]
        rest_occ = tot_occ - i_occ
        rest_vars = tot_vars - i_vars
        oddsr, pval = stats.fisher_exact([[i_vars, rest_vars], [i_occ, rest_occ]])
        vals = [i_vars,rest_vars,i_occ, rest_occ]
        se_logor = 1.96*(math.sqrt(sum(list(map((lambda x: 1/x),vals)))))
        logor = math.log(oddsr)
        df.loc[i,'oddsratio'] = oddsr
        df.loc[i,'log_oddsratio'] = logor
        df.loc[i,'pvalue'] = pval
        df.loc[i,'ci_dist'] = se_logor
    return df
def get_missense_df(aln_in, aln_fmt, variants_df, cons_cols, shenkin_aln, aln_out = None, t = 0.5, get_or = True):
    #variants_aln = generate_subset_aln(aln_in, aln_fmt, variants_df, aln_out)
    variants_aln_info = calculate_shenkin(aln_in, aln_fmt, t)
    variants_aln_info = variants_aln_info[variants_aln_info.occ_pct >= t]
    cons_cols = get_cons_cols(aln_in, aln_fmt, t = t)
    vars_df = pd.DataFrame(variants_df.alignment_column.value_counts().reindex(cons_cols, fill_value=0).sort_index()).reset_index()
    
    vars_df.index = range(1, len(vars_df)+1)
    vars_df.columns = ["col", "variants"]
    variants_aln_info = variants_aln_info[variants_aln_info.col.isin(cons_cols)]
    
    variants_aln_info.index = range(1, len(variants_aln_info)+1)
    
    merged = pd.merge(variants_aln_info, vars_df, on = "col", how = 'left')
    
    merged.index = range(1, len(merged)+1)
    merged.variants = merged.variants + 1
    merged["shenkin"] = shenkin_aln["shenkin"]
    merged["norm_shenkin_rel"] = shenkin_aln["norm_shenkin_rel"] # REPLACES ONLY HUMAN DIVERGENCE SCORES BY ALL SPECIES, FULL ALIGNMENT PROVIDES A BETTER MEASURE FOR THE OVERALL DIVERGENCE
    merged["norm_shenkin_abs"] = shenkin_aln["norm_shenkin_abs"]
    
    if get_or == True:
        merged_or = get_OR(merged)
        return merged_or
    else:
        return merged
aln_out_human_missense_variants = "/homes/2414054/varalign_folder/Altered_Alignment/Changing_Alignment/Human_Alignment127_PFAM.sto"

missense_variants_df = get_missense_df(
    aln_in, aln_fmt, human_missense_variants, cons_cols,
    shenkin_aln_filt, aln_out_human_missense_variants, t = 0.50
) # CALCULATES ENRICHMENT IN MISSENSE VARIANTS AND ASSOCIATED P-VALUE AND 95% CI
print(missense_variants_df)


synomous_variants_df = get_missense_df(
    aln_in, aln_fmt, human_synomous_variants, cons_cols,
    shenkin_aln_filt, aln_out_human_missense_variants, t = 0.50
) # CALCULATES ENRICHMENT IN MISSENSE VARIANTS AND ASSOCIATED P-VALUE AND 95% CI
print(missense_variants_df)


In [ ]:
len(cons_cols)

In [ ]:
def add_miss_class(df, cons_col = "shenkin", thresholds = [30, 70], colours = ["royalblue", "seagreen", "grey", "purple", "orange"]):
    for i in df.index:          
        if df.loc[i, cons_col] <= thresholds[0] and df.loc[i, "log_oddsratio"] < 0:
            df.loc[i,"miss_class"] = "CMD"
        elif df.loc[i, cons_col] <= thresholds[0] and df.loc[i, "log_oddsratio"] > 0:
            df.loc[i,"miss_class"] = "CME"
        elif df.loc[i, cons_col] >= thresholds[1] and df.loc[i, "log_oddsratio"] < 0:
            df.loc[i,"miss_class"] = "UMD"
        elif df.loc[i, cons_col] >= thresholds[1] and df.loc[i, "log_oddsratio"] > 0:
            df.loc[i,"miss_class"] = "UME"
        else:
            df.loc[i,"miss_class"] = "None"
           
    coloring = {
        "CMD": colours[0],
        "CME": colours[1],
        "UMD": colours[3],
        "UME": colours[4],
        "None": colours[2]
    }
    df["miss_color"] =  df.miss_class.map(coloring) 
    return df
print(missense_variants_df)
missense_variants_df = add_miss_class(missense_variants_df, cons_col = "norm_shenkin_rel", thresholds = [25,75]) # ADDS CLASSIFICATION ACCORDING TO MES AND NORMALISED SHENKIN
missense_variants_df

synomous_variants_df= add_miss_class(synomous_variants_df, cons_col = "norm_shenkin_rel", thresholds = [25,75]) # ADDS CLASSIFICATION ACCORDING TO MES AND NORMALISED SHENKIN


In [ ]:
def plot_shenkin_logOR(df, strat_column, color_column, cons_col = "shenkin", thresholds = [30,70], out = None, markers = ["h", ".", "s", "*", "^"], colors = ["darkred", "tan", "green", "darkblue", "yellow"], label = True, error = True):
    plt.figure(figsize=(140,100))
    classes_df = sorted(df[strat_column].unique().tolist())
    #print(classes_df)
    i = 0
    for c in classes_df:
        #print(c)
        df_c = df[df[strat_column] == c]
        color = df_c[color_column].unique().tolist()[0]
        plt.scatter(df_c[cons_col], df_c.log_oddsratio,  marker = markers[i], c = colors[i], label = c,  s = 5000  ) #, edgecolor = 'black', linewidth = 10)
        if error == True:
            plt.errorbar(df_c[cons_col],df_c.log_oddsratio, yerr=df_c.ci_dist, c = colors[i], linewidth = 1, linestyle="None", capsize = 1.0, capthick = 1)
        i += 1
    if label == True:
        for x, y, z in zip(df[cons_col], df.log_oddsratio, df.index):
            label = "{:d}".format(z)
            plt.annotate(label, # this is the text
                            (x,y), # this is the point to label
                            textcoords="offset points", # how to position the text
                            xytext=(-75,75), # distance from text to points (x,y)
                            ha='right',
                            fontsize = 60) # horizontal alignment can be left, right or center
    #plt.title("Missense enrichment score relative to sequence divergence", fontsize = 200, pad = 140)
    plt.xlabel("Normalised Shenkin divergence score", fontsize = 180, labelpad = 60)
    plt.ylabel("MES", fontsize = 180, labelpad = 60)
    plt.tick_params(axis= 'both' , which = 'major', pad = 30, width = 15, length = 50, labelsize = 160)
    #plt.yticks(np.arange(-0.6,0.3,0.1))
    plt.xticks(np.arange(0,105,5))
    plt.xlim(-5,105)
    #plt.ylim(-1.5,1.25)
    plt.axhline(color = "black", linewidth = 10, linestyle = '--')
    plt.axvline(x= thresholds[0], color = "black", linewidth = 10, linestyle = '--')
    plt.axvline(x= thresholds[1], color = "black", linewidth = 10, linestyle = '--')
    legend = plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize = 140)
    legend.get_frame().set_linewidth(10)
    legend.get_frame().set_edgecolor("black")
    for legobj in legend.legendHandles:
        legobj.set_linewidth(7.5)
    if out != None:
        plt.savefig(out)
    plt.show()



significant_missense_variants_df = missense_variants_df.loc[missense_variants_df['pvalue'] < 0.05]
print(len(significant_missense_variants_df))
plot_shenkin_logOR(
    significant_missense_variants_df, "miss_class", "miss_color",
    markers = ["D", "s", "o", "h", "^"],
    colors = ["royalblue", "seagreen", "grey", "purple", "orange"],
    cons_col = "norm_shenkin_rel", thresholds = [25,75])


In [ ]:
def get_tot_occ_res(occ_dict, n_cols, contact_map = "intra", t = 0):
    res_occ = {}
    for i in list(occ_dict.keys()):
        if i[0] not in res_occ:
            res_occ[i[0]] = 0
        if contact_map == "intra":
            if abs(i[1] - i[0]) > t:
                res_occ[i[0]] += occ_dict[(i[0],i[0])]
    for x in range(1,n_cols,1):
        Found = "False"
        for y in res_occ.keys():
            if x == y:
                Found = "True"
        if Found == "False":
            res_occ[x] = 0
    res_occ = dict(sorted(res_occ.items()))
    return res_occ


        



def get_tot_cons_res(cons_df, contact_map = "intra", t = 0):
    res_cons = {}
    idx = cons_df.index.tolist()
    for i in idx:
        if i not in res_cons:
            res_cons[i] = 0
        for j in idx:
            if contact_map == "intra":
                
                if abs(j - i) > t:
                    res_cons[i] += cons_df.loc[i,j]
            elif contact_map == "inter":
                if i == j:
                    res_cons[i] += cons_df.loc[i,j]
                else:
                    res_cons[i] += (cons_df.loc[i,j] + cons_df.loc[j,i])         
    return res_cons

def get_OR(df):
    df.contacts = df.contacts + 1 #pseudocount
    tot_occ = sum(df.occ)
    tot_cons = sum(df.contacts)
    for i in df.index:
        i_occ = df.loc[i,"occ"]
        i_cons = df.loc[i,'contacts']

        rest_occ = tot_occ - i_occ
        rest_cons = tot_cons - i_cons
        
        oddsr, pval = stats.fisher_exact([[i_cons, rest_cons], [i_occ, rest_occ]])
        vals = [i_cons,rest_cons,i_occ, rest_occ]
        se_logor = 1.96*(math.sqrt(sum(list(map((lambda x: 1/x),vals)))))
        logor = math.log(oddsr)
        df.loc[i,'oddsratio'] = oddsr
        df.loc[i,'log_oddsratio'] = logor
        df.loc[i,'pvalue'] = pval
        df.loc[i,'ci_dist'] = se_logor
    return df




def get_OR_from_cons(res_cons, res_occ):
    cons = list(res_cons.values())
    occ = list(res_occ.values())
    df = pd.DataFrame(list(zip(occ, cons)), columns =['occ', 'contacts'])
    df.index = range(1,len(df)+1)
    df = get_OR(df)
    return df



def add_colours(df, missense_variants_df ):
    jioner = missense_variants_df[['miss_class','miss_color']].copy()
    df2 = pd.merge(df, jioner, left_index=True, right_index=True)
    return df2


def fisher_exact_test(df,x):
    
    cmd_cons = df[df.miss_class == x].contacts.sum()
    cmd_occ = df[df.miss_class == x].occ.sum()
    rest_cons = df.contacts.sum() - cmd_cons
    rest_occ = df.occ.sum() - cmd_occ
    oddsr1, pval1 = stats.fisher_exact([[cmd_cons, rest_cons], [cmd_occ, rest_occ]]) # CMDs ARE NOT ENRICHED IN INTER-REPEAT CONTACTS

    cmd_cons = df[df.miss_class == x].contacts.sum()
    cmd_occ = df[df.miss_class == x].occ.sum()
    rest_cons = df[df.miss_class != x].contacts.sum()
    rest_occ = df[df.miss_class != x].occ.sum()
    oddsr2, pval2 = stats.fisher_exact([[cmd_cons, rest_cons], [cmd_occ, rest_occ]]) # CMDs ARE ENRICHED IN INTRA-REPEAT CONTACTS RELATIV
    if oddsr1 == oddsr2 and pval1 == pval2:
        print(x, oddsr1, pval1)
    else:
        print(x, oddsr1, pval1)
        print()
        print(x, oddsr2, pval2)
    

In [ ]:
for y in (5,0):
    intra_res_occ  = get_intra_cons_occ(df_rf, cons_cols_eq) # GETS COVERAGE OF EACH INTRA-REPEAT POSITION PAIR IN OUR STRUCTURAL DATASET
    
    occ_intra = get_tot_occ_res(intra_res_occ, len(intra), contact_map = "intra", t = y) # CALCULATES THE ABSOLUTE STRUCTURAL COVERAGE PER DOMAIN POSITION
    cons_intra = get_tot_cons_res(intra, contact_map = "intra", t = y) # CALCULATES THE ABSOLUTE NUMBER OF CONTACTS PER DOMAIN POSITION
    enrichment_df_intra = get_OR_from_cons(cons_intra, occ_intra) # CALCULATES OR, LOG(OR), P-VALUE AND 95% CI FOR THE ENRICHMENT IN CONTACTS
    enrichment_df_intra = add_colours(enrichment_df_intra,missense_variants_df) # ADDS MISSENSE ENRICHMENT CLASS
    for x in list(enrichment_df_intra.miss_class.unique()):
        fisher_exact_test(enrichment_df_intra,x)



In [ ]:
enrichment_df_intra

In [ ]:
def plot_cons_enrichment(df, contact_map = "Intra", legend_title = None, class_col = None, color_col = None, out = None):
    plt.figure(figsize=(140,60))
    plt.rcParams.update({"axes.linewidth": 5})
    #plt.title("Enrichment contacts per residue position", fontsize = 140, pad = 100)
    plt.xlabel("Occupancy residue position", fontsize = 120, labelpad = 100)
    plt.ylabel("Contacts enrichment score".format(contact_map), fontsize = 120, labelpad = 80)
    plt.tick_params(axis= 'both' , which = 'major', pad = 60, width = 5, length = 30, labelsize = 80)
    df_len = len(df)+3
    print(df_len)
    plt.xticks(np.arange(0,df_len ,20))
    plt.xlim(0,df_len)
    plt.axhline(y = 0, color = "black", linewidth = 5)
    lines_range = list(np.arange(0,df_len,5))
    for point in lines_range:
        plt.axvline(x= point, color = "grey", linestyle = '--', linewidth = 5)
    if class_col != None and color_col != None:
        classes = list(df[class_col].unique())
        for c in classes:
            df_c = df[df[class_col] == c]
            color = df_c[color_col].unique().tolist()[0]
            plt.scatter(df_c.index, df_c.log_oddsratio, c = color, label = c, s = 5000, edgecolor = 'black', linewidth = 10)
            #plt.errorbar(df_c.index,df_c.log_oddsratio, yerr=df_c.ci_dist, c = color, linewidth = 7.5, linestyle="None", capsize = 35.0, capthick = 7.5)
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize = 80)#, title = legend_title, title_fontsize = 120)
    else:
        plt.scatter(df.index, df.log_oddsratio, s = 5000, edgecolor = 'black', linewidth = 7.5)
        #plt.errorbar(df.index,df.log_oddsratio, yerr=df.ci_dist, linewidth = 7.5, linestyle="None", capsize = 35.0, capthick = 7.5)
    if out != None:
        plt.savefig(out)
    plt.show()


print(len(enrichment_df_intra))
plot_cons_enrichment(enrichment_df_intra, contact_map = "Intra", class_col = "miss_class", color_col = "miss_color")

In [ ]:
def plot_cons_enrichment(df, contact_map = "Intra", legend_title = None, class_col = None, color_col = None, out = None):
    plt.figure(figsize=(140,60))
    plt.rcParams.update({"axes.linewidth": 5})
    #plt.title("Enrichment contacts per residue position", fontsize = 140, pad = 100)
    plt.xlabel("Occupancy residue position", fontsize = 120, labelpad = 100)
    plt.ylabel("Contacts enrichment score".format(contact_map), fontsize = 120, labelpad = 100)
    plt.tick_params(axis= 'both' , which = 'major', pad = 60, width = 5, length = 30, labelsize = 80)
    plt.xticks(np.arange(0,401,20))
    plt.xlim(0,401)
    plt.axhline(y = 0, color = "black", linewidth = 5)
    lines_range = list(np.arange(0,401,5))
    for point in lines_range:
        plt.axvline(x= point, color = "grey", linestyle = '--', linewidth = 5)
    if class_col != None and color_col != None:
        classes = list(df[class_col].unique())
        for c in classes:
            df_c = df[df[class_col] == c]
            color = df_c[color_col].unique().tolist()[0]
            plt.scatter(df_c.index, df_c.log_oddsratio, c = color, label = c, s = 5000, edgecolor = 'black', linewidth = 10)
            #plt.errorbar(df_c.index,df_c.log_oddsratio, yerr=df_c.ci_dist, c = color, linewidth = 7.5, linestyle="None", capsize = 35.0, capthick = 7.5)
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize = 80)#, title = legend_title, title_fontsize = 120)
    else:
        plt.scatter(df.index, df.log_oddsratio, s = 5000, edgecolor = 'black', linewidth = 7.5)
        plt.errorbar(df.index,df.log_oddsratio, yerr=df.ci_dist, linewidth = 7.5, linestyle="None", capsize = 35.0, capthick = 7.5)
    if out != None:
        plt.savefig(out)
    plt.show()



significant_df = enrichment_df_intra.loc[enrichment_df_intra['pvalue'] < 0.05]
plot_cons_enrichment(significant_df, contact_map = "Intra", class_col = "miss_class", color_col = "miss_color")

In [ ]:
active_site = [97,98,99,283,284,285]
for x in active_site:
    print(x,enrichment_df_intra.miss_class[x], enrichment_df_intra.miss_color[x], enrichment_df_intra.pvalue[x], enrichment_df_intra.ci_dist[x])

#print(x,significant_df.miss_class[284], significant_df.miss_color[284], significant_df.pvalue[284], significant_df.ci_dist[284])

In [ ]:
active_site = [97,98,99,283,284,285]

for x in active_site:
    secondary = ""
    
    if ss_class_df.loc[x].p_helix > ss_class_df.loc[x].p_strand     and ss_class_df.loc[x].p_helix > ss_class_df.loc[x].p_coil      :
        print(ss_class_df.loc[x].p_helix)
        secondary = "helix"  
    elif ss_class_df.loc[x].p_strand     > ss_class_df.loc[x].p_helix and ss_class_df.loc[x].p_strand     > ss_class_df.loc[x].p_coil      :    
        print(ss_class_df.loc[x].p_strand)
        secondary = "strand" 
    elif ss_class_df.loc[x].p_coil > ss_class_df.loc[x].p_helix and ss_class_df.loc[x].p_coil > ss_class_df.loc[x].p_strand    : 
        print(ss_class_df.loc[x].p_coil)   
        secondary = "loop" 
    elif secondary == "":
        if ss_class_df.loc[x].p_helix == ss_class_df.loc[x].p_strand:
            secondary =  "helix, strand"
            print(ss_class_df.loc[x].p_helix, ss_class_df.loc[x].p_strand)  
        elif ss_class_df.loc[x].p_helix == ss_class_df.loc[x].p_coil:
            secondary =  "helix, loop"
            print(ss_class_df.loc[x].p_helix, ss_class_df.loc[x].p_coil)  
        elif ss_class_df.loc[x].p_strand == ss_class_df.loc[x].p_coil:
            secondary =  "strand, loop"    
            print(ss_class_df.loc[x].p_strand, ss_class_df.loc[x].p_coil)  
        
        
    print(secondary)


In [ ]:
##### Median RSA for active site
active_site = [97,98,99,283,284,285]
for x in active_site:
    print(vs[x])

In [ ]:
active_site = [97,98,99,283,284,285]

for x in active_site:
    for y in range(1,len(intra_norm),1):
        if intra_norm.loc[x,y] > 0.9:
            print(x, y, intra_norm.loc[x,y])
    print("\n")
    

In [ ]:
### Active Site
active_site = [97,98,99,283,284,285]
for x in active_site:
    print(shenkin_aln_filt.norm_shenkin_rel[x])

In [ ]:
active_site = [97,98,99,283,284,285]

for x in active_site:
    secondary = ""
    dictss = {}
    dictss ={"p_a_helix": ss_df.loc[x].p_a_helix,
    "p_b_bridge" : ss_df.loc[x].p_b_bridge ,
    "p_strand" : ss_df.loc[x].p_strand,
    "p_3_10_helix" : ss_df.loc[x].p_3_10_helix,
    "p_pi_helix" : ss_df.loc[x].p_pi_helix,
    "p_turn": ss_df.loc[x].p_turn,
    "p_bend" : ss_df.loc[x].p_bend,
    "p_coil" : ss_df.loc[x].p_coil}
    #print(dictss)
    maxim = 0
    highest = ""
    for y in dictss:
        
        if dictss[y] > maxim:
            maxim = dictss[y]
            secondary = y

    print(secondary, maxim)

In [ ]:
def get_aln_len(aln_in, fmt_in):
    aln = Bio.AlignIO.read(aln_in, fmt_in)
    return aln.get_alignment_length()

def get_ppis(df, aln_len, col_mask):
    df_filt = df.dropna(subset=['UniProt_dbAccessionId_A', 'UniProt_dbAccessionId_B'])
    df_filt = df_filt[(df_filt.UniProt_dbAccessionId_A != df_filt.UniProt_dbAccessionId_B)&(df_filt.interaction_type == 'Protein-Protein')]
    get_struc_info(df_filt)
    df_reps = df_filt.groupby("SOURCE_ID_A")
    cons_norm = pd.DataFrame(0, index = range(1, aln_len+1), columns = ["contacts"])
    for repeat, row in df_reps: # REPEAT
        df_strucs = row.groupby("PDB_dbAccessionId_A")
        rep_cons = pd.DataFrame(0, index = range(1, aln_len+1), columns = ["cons"])
        for struc, row in df_strucs: # STRUCTURE
            df_chains = row.groupby("PDB_dbChainId_A")
            struc_cons = pd.DataFrame(0, index = range(1, aln_len+1), columns = ["cons"])
            for chain, row in df_chains: # CHAIN
                chain_cons = pd.DataFrame(row.groupby("Alignment_column_A")["interaction_type"].count()).reindex(range(1, aln_len+1)).fillna(0)
                struc_cons.cons = struc_cons.cons + chain_cons.interaction_type
            struc_cons.cons = struc_cons.cons.map(int).map(lambda x: 1 if x > 0 else 0)
            rep_cons.cons = rep_cons.cons + struc_cons.cons
        rep_cons.cons = rep_cons.cons.map(int).map(lambda x: 1 if x > 0 else 0)
        cons_norm.contacts = cons_norm.contacts + rep_cons.cons
    cons_norm = cons_norm[cons_norm.index.isin(col_mask)]
    cons_norm.index = range(1,len(col_mask)+1)
    return cons_norm



def get_struc_res_occ(structure_table, col_mask, interaction_mask = None): #returns a dataframe with the occupancy of every alignment column in structure
    if interaction_mask != None:
        structure_table_interaction = structure_table[(structure_table.UniProt_dbAccessionId_A != structure_table.UniProt_dbAccessionId_B)&(structure_table.interaction_type == interaction_mask)]
        structures_mask = structure_table_interaction.PDB_dbAccessionId_A.unique().tolist()
        structure_table = structure_table[structure_table.PDB_dbAccessionId_A.isin(structures_mask)]
    table_a = structure_table[['UniProt_dbAccessionId_A','UniProt_dbResNum_A','Alignment_column_A']]
    table_a = table_a.drop_duplicates(['UniProt_dbAccessionId_A','UniProt_dbResNum_A']).dropna()
    table_a.Alignment_column_A = table_a.Alignment_column_A.astype(float).astype(int)
    table_a = table_a.rename(columns={'UniProt_dbAccessionId_A':'UniProt_dbAccessionId','UniProt_dbResNum_A':'UniProt_dbResNum','Alignment_column_A':'Alignment_column'})
    
    table_b = structure_table[['UniProt_dbAccessionId_B','UniProt_dbResNum_B','Alignment_column_B']]
    table_b = table_b.drop_duplicates(['UniProt_dbAccessionId_B','UniProt_dbResNum_B']).dropna()
    table_b.Alignment_column_B = table_b.Alignment_column_B.astype(float).astype(int)
    table_b = table_b.rename(columns={'UniProt_dbAccessionId_B':'UniProt_dbAccessionId','UniProt_dbResNum_B':'UniProt_dbResNum','Alignment_column_B':'Alignment_column'})

    table_ab = pd.concat([table_a,table_b])
    table_ab = table_ab.drop_duplicates(['UniProt_dbAccessionId','UniProt_dbResNum'])
    res_occ = pd.DataFrame(table_ab.Alignment_column.value_counts())
    res_occ = res_occ[res_occ.index.isin(col_mask)]
    
    
    res_occ.reset_index(inplace = True)
    res_occ = res_occ.rename(columns={'Alignment_column': 'occ'})
    res_occ.index.name = 'aln_col'
    print(res_occ)
    print(col_mask)
    #res_occ = res_occ[res_occ.index.isin(col_mask)]
    res_occ = res_occ.sort_values(by=['aln_col'])#
    
    res_occ.index = range(1,len(res_occ)+1)
    return res_occ



def get_OR(df):
    df.contacts = df.contacts + 1 #pseudocount
    tot_occ = sum(df.occ)
    tot_cons = sum(df.contacts)
    for i in df.index:
        i_occ = df.loc[i,"occ"]
        i_cons = df.loc[i,'contacts']
        rest_occ = tot_occ - i_occ
        rest_cons = tot_cons - i_cons
        oddsr, pval = stats.fisher_exact([[i_cons, rest_cons], [i_occ, rest_occ]])
        vals = [i_cons,rest_cons,i_occ, rest_occ]
        se_logor = 1.96*(math.sqrt(sum(list(map((lambda x: 1/x),vals)))))
        logor = math.log(oddsr)
        df.loc[i,'oddsratio'] = oddsr
        df.loc[i,'log_oddsratio'] = logor
        df.loc[i,'pvalue'] = pval
        df.loc[i,'ci_dist'] = se_logor
    return df



def get_struc_res_occ(structure_table, col_mask, interaction_mask = None): #returns a dataframe with the occupancy of every alignment column in structure
    print(interaction_mask)
    if interaction_mask != None:
        structure_table_interaction = structure_table[(structure_table.UniProt_dbAccessionId_A != structure_table.UniProt_dbAccessionId_B)&(structure_table.interaction_type == interaction_mask)]
        structures_mask = structure_table_interaction.PDB_dbAccessionId_A.unique().tolist()
        structure_table = structure_table[structure_table.PDB_dbAccessionId_A.isin(structures_mask)]
    table_a = structure_table[['UniProt_dbAccessionId_A','UniProt_dbResNum_A','Alignment_column_A']]
    table_a = table_a.drop_duplicates(['UniProt_dbAccessionId_A','UniProt_dbResNum_A']).dropna()
    table_a.Alignment_column_A = table_a.Alignment_column_A.astype(float).astype(int)
    table_a = table_a.rename(columns={'UniProt_dbAccessionId_A':'UniProt_dbAccessionId','UniProt_dbResNum_A':'UniProt_dbResNum','Alignment_column_A':'Alignment_column'})
    
    table_b = structure_table[['UniProt_dbAccessionId_B','UniProt_dbResNum_B','Alignment_column_B']]
    table_b = table_b.drop_duplicates(['UniProt_dbAccessionId_B','UniProt_dbResNum_B']).dropna()
    table_b.Alignment_column_B = table_b.Alignment_column_B.astype(float).astype(int)
    table_b = table_b.rename(columns={'UniProt_dbAccessionId_B':'UniProt_dbAccessionId','UniProt_dbResNum_B':'UniProt_dbResNum','Alignment_column_B':'Alignment_column'})

    table_ab = pd.concat([table_a,table_b])
    table_ab = table_ab.drop_duplicates(['UniProt_dbAccessionId','UniProt_dbResNum'])
    res_occ = pd.DataFrame(table_ab.Alignment_column.value_counts())
    res_occ.reset_index(inplace = True)
    
    
    
    res_occ = res_occ.rename(columns={'index':'aln_col','Alignment_column':'occ'})
    
    res_occ.sort_index(inplace=True)
    col_mask = range(1,len(col_mask)+1, 1)
    res_occ = res_occ[res_occ.index.isin(col_mask)]

    

    return res_occ


In [ ]:

aln_len_allsp = get_aln_len('/homes/2414054/varalign_folder/Altered_Alignment/Changing_Alignment/Alignment127_PFAM.sto', 'stockholm') # ALIGNMENT LENGTH (NUMBER OF COLUMNS)
cons_norm_allsp = get_ppis(df_allsp_rsrz_filt, aln_len_allsp, cons_cols_allsp) # PPI CONTACTS PER POSITION


res_occ_allsp = get_struc_res_occ(df_allsp, cons_cols_allsp, "Protein-Protein") # STRUCTURAL COVERAGE OF EACH POSITION IN OUR DATASET
contacts_allsp_Protein = pd.concat([res_occ_allsp, cons_norm_allsp], axis = 1)
contacts_allsp_Protein = get_OR(contacts_allsp_Protein) # CALCULATES ENRICHMENT IN PPIs AS WELL AS P-VALUE AND 95% CI
contacts_allsp_Protein = add_colours(contacts_allsp_Protein,missense_variants_df) # ADDS MISSENSE ENRICHMENT CLASS
contacts_allsp_Protein[contacts_allsp_Protein['contacts'] > 0]


In [ ]:
def add_colours(df, missense_variants_df ):
    jioner = missense_variants_df[['miss_class','miss_color']].copy()
    df2 = pd.merge(df, jioner, left_index=True, right_index=True)
    return df2

def plot_ppi_enrichment(df, Yaxis_name, legend_title = None, class_col = None, color_col = None, out = None, significant=False, error=True):
    if significant == True:
        df = df.loc[df['pvalue'] < 0.05]

    plt.figure(figsize=(180,80))
    #plt.title("Enrichment in PPIs per residue position", fontsize = 140, pad = 100)
    plt.xlabel("Domain position", fontsize = 160, labelpad = 100)
    plt.ylabel(Yaxis_name, fontsize = 160, labelpad = 100)
    plt.tick_params(axis= 'both' , which = 'major', pad = 60, width = 15, length = 50, labelsize = 140)
    plt.xticks(np.arange(1,len(df),20))
    plt.xlim(0,len(df))
    plt.axhline(y = 0, color = "black", linewidth = 7.5)
    lines_range = list(np.arange(5,len(df),5))
    for point in lines_range:
        plt.axvline(x= point, color = "grey", linestyle = '--', linewidth = 7.5)
    plt.xticks(np.arange(0,len(df),20))
    if class_col != None and color_col != None:
        classes = list(df[class_col].unique())
        for c in classes:
            df_c = df[df[class_col] == c]
            color = df_c[color_col].unique().tolist()[0]
            plt.scatter(df_c.index, df_c.log_oddsratio, c = color, label = c, s = 10000, edgecolor = 'black', linewidth = 10)
            if error == True:
                plt.errorbar(df_c.index,df_c.log_oddsratio, yerr=df_c.ci_dist, c = color, linewidth = 10, linestyle="None", capsize = 35.0, capthick = 7.5)
        legend = plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize = 120)#, title = legend_title, title_fontsize = 120)
        legend.get_frame().set_linewidth(10)
        legend.get_frame().set_edgecolor("black")
        for legobj in legend.legendHandles:
            legobj.set_linewidth(7.5)
    else:
        plt.scatter(df.index, df.log_oddsratio, s = 7500, edgecolor = 'black', linewidth = 10)
        plt.errorbar(df.index,df.log_oddsratio, yerr=df.ci_dist, linewidth = 10, linestyle="None", capsize = 35.0, capthick = 7.5)
    if out != None:
        plt.savefig(out)
    plt.show()






plot_ppi_enrichment(contacts_allsp_Protein,Yaxis_name="PPIES",legend_title= "Surface", class_col =  "miss_class", color_col= "miss_color", significant=False, error=False)

In [ ]:
res_occ_allsp = get_struc_res_occ(df_allsp, cons_cols_allsp, "Protein-Ligand") # STRUCTURAL COVERAGE OF EACH POSITION IN OUR DATASET
contacts_allsp_Ligand = pd.concat([res_occ_allsp, cons_norm_allsp], axis = 1)
contacts_allsp_Ligand = get_OR(contacts_allsp_Ligand) # CALCULATES ENRICHMENT IN PPIs AS WELL AS P-VALUE AND 95% CI
contacts_allsp_Ligand = add_colours(contacts_allsp_Ligand,missense_variants_df) # ADDS MISSENSE ENRICHMENT CLASS
plot_ppi_enrichment(contacts_allsp_Ligand,Yaxis_name="PPIES, Ligand",legend_title= "Surface", class_col =  "miss_class", color_col= "miss_color", significant=False, error=False)

In [ ]:
contacts_allsp_Ligand

In [ ]:
def fisher_exact_test_prot_lig(df,typess ="Protein-Protein"):
    
    for x in list(df.miss_class.unique()):
        ppis_cons = df[df.miss_class == x].contacts.sum()
        ppis_occ = df[df.miss_class == x].occ.sum()
        rest_cons = df.contacts.sum() - ppis_cons
        rest_occ = df.occ.sum() - ppis_occ
        
        oddsratio, pvalue = stats.fisher_exact([[ppis_cons, rest_cons], [ppis_occ, rest_occ]]) # BURIED RESIDUES DEPLETED IN PPIS RELATIVE TO SURFACE RESIDUES
        #print(x, math.log(oddsratio), pvalue)
        print(x,typess, oddsratio, pvalue)

fisher_exact_test_prot_lig(contacts_allsp_Protein, typess ="Protein-Protein")
fisher_exact_test_prot_lig(contacts_allsp_Ligand, typess ="Protein-Ligand")

In [ ]:
x = synomous_variants_df['norm_shenkin_rel']
y = synomous_variants_df['variants']

plt.figure(figsize=(180,80))

a, b = np.polyfit(x, y, 1)
plt.scatter(x, y,s = 500, edgecolor = 'black', linewidth = 1)
plt.plot(x, a*x+b) 
plt.xlabel('norm_shenkin_rel')
plt.ylabel('variants')
plt.xlabel("norm_shenkin_rel", fontsize = 160, labelpad = 100)
plt.ylabel("variants", fontsize = 160, labelpad = 100)
plt.tick_params(axis= 'both' , which = 'major', pad = 60, width = 15, length = 50, labelsize = 140)
plt.xticks(np.arange(0,100,10))
plt.xlim(0,100)

plt.show()




In [ ]:
missense_variants_df['norm_shenkin_rel'].unique()

In [ ]:
missense_variants_df